In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('E:\DM\kmeans_data\data.csv')
label = pd.read_csv('E:\DM\kmeans_data\label.csv')

In [4]:
def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

def cosine_distance(a, b):
    cosine_similarity = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return 1 - cosine_similarity

def jaccard_distance(a, b):
    intersection = np.minimum(a, b).sum()
    union = np.maximum(a, b).sum()
    jaccard_similarity = intersection / union
    return 1 - jaccard_similarity

In [5]:
def assign_clusters(X, centroids, distance_func):
    clusters = []
    for x in X:
        distances = [distance_func(x, centroid) for centroid in centroids]
        closest = np.argmin(distances)
        clusters.append(closest)
    return np.array(clusters)

def update_centroids(X, clusters, k):
    new_centroids = np.array([X[clusters == i].mean(axis=0) for i in range(k)])
    return new_centroids

In [6]:
def calculate_sse(X, centroids, clusters, distance_func):
    sse = 0
    for i, centroid in enumerate(centroids):
        sse += np.sum([distance_func(x, centroid) ** 2 for x in X[clusters == i]])
    return sse


In [7]:
def kmeans(X, k, distance_func, max_iters=100):
    indices = np.random.choice(len(X), k, replace=False)
    centroids = X[indices]

    for _ in range(max_iters):
        clusters = assign_clusters(X, centroids, distance_func)
        new_centroids = update_centroids(X, clusters, k)

        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    sse = calculate_sse(X, centroids, clusters, distance_func)
    return centroids, clusters, sse


In [11]:
X = data.to_numpy()
k = len(np.unique(label))

labels =  label.to_numpy()
labels =  labels.flatten()

centroids_euclidean, clusters_euclidean, sse_euclidean = kmeans(X, k, euclidean_distance)
centroids_cosine, clusters_cosine, sse_cosine = kmeans(X, k, cosine_distance)
centroids_jaccard, clusters_jaccard, sse_jaccard = kmeans(X, k, jaccard_distance)

print(f"SSE Euclidean: {sse_euclidean}, SSE Cosine: {sse_cosine}, SSE Jaccard: {sse_jaccard}")

SSE Euclidean: 25371934362.58509, SSE Cosine: 687.405925556465, SSE Jaccard: 3660.810730806501


In [13]:
def label_clusters_with_majority_vote(clusters, actual_labels):
    
    unique_clusters = np.unique(clusters)
    cluster_labels = {}
    for cluster in unique_clusters:
        # Find the indices of points in this cluster
        indices = np.where(clusters == cluster)[0]
        # Find the actual labels of these points
        labels_in_cluster = actual_labels[indices]
        # Determine the majority label
        majority_label = np.bincount(labels_in_cluster).argmax()
        cluster_labels[cluster] = majority_label

    # Map the cluster assignments to the predicted labels
    predicted_labels = np.array([cluster_labels[cluster] for cluster in clusters])

    return cluster_labels, predicted_labels

In [14]:
def calculate_accuracy(predicted_labels, actual_labels):
    correct_predictions = np.sum(predicted_labels == actual_labels)
    accuracy = correct_predictions / len(actual_labels)
    return accuracy


In [17]:
actual_labels_array = np.array(labels)

_, predicted_labels_euclidean = label_clusters_with_majority_vote(clusters_euclidean, actual_labels_array)
accuracy_euclidean = calculate_accuracy(predicted_labels_euclidean, actual_labels_array)

_, predicted_labels_cosine = label_clusters_with_majority_vote(clusters_cosine, actual_labels_array)
accuracy_cosine = calculate_accuracy(predicted_labels_cosine, actual_labels_array)

_, predicted_labels_jaccard = label_clusters_with_majority_vote(clusters_jaccard, actual_labels_array)
accuracy_jaccard = calculate_accuracy(predicted_labels_jaccard, actual_labels_array)

print(f"Accuracy Euclidean: {accuracy_euclidean}")
print(f"Accuracy Cosine: {accuracy_cosine}")
print(f"Accuracy Jaccard: {accuracy_jaccard}")

Accuracy Euclidean: 0.590959095909591
Accuracy Cosine: 0.5777577757775778
Accuracy Jaccard: 0.6051605160516051


In [18]:
import time


def kmeans_with_stop_criteria(X, k, distance_func, max_iters=500):
    indices = np.random.choice(len(X), k, replace=False)
    centroids = X[indices]
    prev_sse = float('inf')  

    start_time = time.time() 

    for iter_count in range(max_iters):
        clusters = assign_clusters(X, centroids, distance_func)
        new_centroids = update_centroids(X, clusters, k)
        sse = calculate_sse(X, new_centroids, clusters, distance_func)

        if np.all(centroids == new_centroids) or sse > prev_sse:
            break  

        centroids = new_centroids
        prev_sse = sse

    end_time = time.time()  
    elapsed_time = end_time - start_time  

    return iter_count + 1, elapsed_time 



X = data.to_numpy()


actual_labels_array = np.array(label).flatten()

stopping_criteria = ["no change in centroid position", "SSE increase", "max iterations"]


for criterion in stopping_criteria:
    print(f"\nStopping Criterion: {criterion}\n")

    for distance_metric, distance_func in [("Euclidean", euclidean_distance), ("Cosine", cosine_distance), ("Jaccard", jaccard_distance)]:
        print(f"{distance_metric}-K-means:")
        iterations, elapsed_time = kmeans_with_stop_criteria(X, len(np.unique(actual_labels_array)), distance_func)
        print(f"Number of iterations: {iterations}, Time: {elapsed_time:.4f} seconds\n")



Stopping Criterion: no change in centroid position

Euclidean-K-means:
Number of iterations: 32, Time: 33.3432 seconds

Cosine-K-means:
Number of iterations: 40, Time: 70.7823 seconds

Jaccard-K-means:
Number of iterations: 38, Time: 62.0698 seconds


Stopping Criterion: SSE increase

Euclidean-K-means:
Number of iterations: 90, Time: 95.9069 seconds

Cosine-K-means:
Number of iterations: 24, Time: 41.6197 seconds

Jaccard-K-means:
Number of iterations: 45, Time: 70.6517 seconds


Stopping Criterion: max iterations

Euclidean-K-means:
Number of iterations: 33, Time: 33.2128 seconds

Cosine-K-means:
Number of iterations: 60, Time: 101.1330 seconds

Jaccard-K-means:
Number of iterations: 16, Time: 25.1969 seconds



In [ ]:
def kmeans_with_stop_criteria(X, k, distance_func, max_iters=100):
    indices = np.random.choice(len(X), k, replace=False)
    centroids = X[indices]
    prev_sse = float('inf')  
    sse_values = [] 

    for iter_count in range(max_iters):
        clusters = assign_clusters(X, centroids, distance_func)
        new_centroids = update_centroids(X, clusters, k)
        sse = calculate_sse(X, new_centroids, clusters, distance_func)
        sse_values.append(sse)  

        if np.all(centroids == new_centroids):
            break 

        if sse > prev_sse:
            break  

        centroids = new_centroids
        prev_sse = sse

    return sse_values, iter_count + 1 


# Define stopping criteria
stopping_criteria = ["no change in centroid position", "SSE increase", "max iterations"]

for criterion in stopping_criteria:
    print(f"\nStopping Criterion: {criterion}\n")

    for distance_metric, distance_func in [("Euclidean", euclidean_distance), ("Cosine", cosine_distance), ("Jaccard", jaccard_distance)]:
        print(f"{distance_metric}-K-means:")
        sse_values, iterations = kmeans_with_stop_criteria(X, len(np.unique(labels)), distance_func)
        print(f"Number of iterations: {iterations}, SSE values: {sse_values}\n")



Stopping Criterion: no change in centroid position

Euclidean-K-means:
Number of iterations: 34, SSE values: [28420211159.216484, 26605178270.717316, 25902138675.83836, 25634085635.19388, 25543402739.539837, 25502018560.09479, 25472454612.31508, 25443270032.890617, 25413416219.97212, 25392079327.983307, 25374127080.14258, 25361632027.257126, 25351038006.688065, 25338523769.787064, 25329223247.134476, 25322589815.72025, 25319883612.07451, 25319450482.183903, 25319156560.229828, 25319065506.401, 25318988697.091522, 25318921010.788, 25318846013.75264, 25318776907.8863, 25318722534.2281, 25318689077.87569, 25318658988.46115, 25318573502.29721, 25318507621.240017, 25318485415.85327, 25318471840.848724, 25318463934.93203, 25318458028.20375, 25318458028.20375]

Cosine-K-means:
Number of iterations: 18, SSE values: [873.7885014716987, 765.6930045880512, 739.1148012765888, 727.2160770443601, 719.1109251600914, 709.3348051858475, 697.0575888463934, 688.1360081384299, 685.0258191160904, 683.3507